In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Activation, RepeatVector
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [7]:
def n(digits=3):
    number = ''
    for i in range(np.random.randint(1, digits + 1)):
        number += np.random.choice(list('0123456789'))
    return int(number)


def padding(chars, maxlen):
    return chars + ' ' * (maxlen - len(chars))

In [13]:
'''
データの生成
'''
N = 20000
N_train = int(N * 0.9)
N_validation = N - N_train

digits = 3  # 最大の桁数
input_digits = digits * 2 + 1  # 例： 123+456
output_digits = digits + 1  # 500+500 = 1000 以上で４桁になる

added = set()
questions = []
answers = []

while len(questions) < N:
    a, b = n(), n()  # 適当な数を２つ生成

    pair = tuple(sorted((a, b)))
    if pair in added:
        continue

    question = '{}+{}'.format(a, b)
    question = padding(question, input_digits)
    answer = str(a + b)
    answer = padding(answer, output_digits)

    added.add(pair)
    questions.append(question)
    answers.append(answer)

chars = '0123456789+ '
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

X = np.zeros((len(questions), input_digits, len(chars)), dtype=np.integer)
Y = np.zeros((len(questions), digits + 1, len(chars)), dtype=np.integer)

for i in range(N):
    for t, char in enumerate(questions[i]):
        X[i, t, char_indices[char]] = 1
    for t, char in enumerate(answers[i]):
        Y[i, t, char_indices[char]] = 1

X_train, X_validation, Y_train, Y_validation = \
    train_test_split(X, Y, train_size=N_train)

In [9]:
from keras.models import load_model

# returns a compiled model
# identical to the previous one
model = load_model('encoder_decoder_model.h5')

In [14]:
'''
予測精度の評価
'''

questions_count = 1000
success_count = 0

for i in range(questions_count):
    index = np.random.randint(0, N_validation)
    question = X_validation[np.array([index])]
    answer = Y_validation[np.array([index])]
    prediction = model.predict_classes(question, verbose=0)

    question = question.argmax(axis=-1)
    answer = answer.argmax(axis=-1)

    q = ''.join(indices_char[i] for i in question[0])
    a = ''.join(indices_char[i] for i in answer[0])
    p = ''.join(indices_char[i] for i in prediction[0])

#    print('-' * 10)
 #   print('Q:  ', q, 'A:  ', p, 'T/F:', end=' ')
#        print('A:  ', p)
#        print('T/F:', end=' ')
    if a == p:
        success_count += 1 
#        print('TRUE')
    else:
        print('Q:  ', q, 'A:  ', p, 'T/F:', end=' ')
        print('FALSE')

accuracy = success_count / questions_count
print("success_count: ",success_count, "/",questions_count)
print("accuracy: ",accuracy)

Q:   31+78   A:   119  T/F: FALSE
Q:   707+201 A:   820  T/F: FALSE
Q:   582+329 A:   811  T/F: FALSE
Q:   858+658 A:   1426 T/F: FALSE
Q:   330+216 A:   556  T/F: FALSE
Q:   926+678 A:   1594 T/F: FALSE
Q:   902+52  A:   955  T/F: FALSE
Q:   839+70  A:   919  T/F: FALSE
Q:   16+416  A:   442  T/F: FALSE
Q:   70+505  A:   585  T/F: FALSE
Q:   173+633 A:   816  T/F: FALSE
Q:   56+869  A:   915  T/F: FALSE
Q:   858+658 A:   1426 T/F: FALSE
Q:   17+929  A:   956  T/F: FALSE
Q:   781+451 A:   1222 T/F: FALSE
Q:   489+624 A:   1123 T/F: FALSE
Q:   97+385  A:   472  T/F: FALSE
Q:   360+538 A:   888  T/F: FALSE
Q:   792+735 A:   1537 T/F: FALSE
Q:   631+21  A:   642  T/F: FALSE
Q:   302+48  A:   340  T/F: FALSE
Q:   789+194 A:   908  T/F: FALSE
Q:   794+554 A:   1438 T/F: FALSE
Q:   586+517 A:   1104 T/F: FALSE
Q:   797+377 A:   1264 T/F: FALSE
Q:   17+929  A:   956  T/F: FALSE
Q:   844+534 A:   1388 T/F: FALSE
Q:   579+19  A:   588  T/F: FALSE
Q:   459+573 A:   1023 T/F: FALSE
Q:   279+753 A